In [3]:
library(tidyverse)
library(repr)
library(tidymodels)

# data <- read_delim("processed.switzerland.csv", delim = ",", col_names = FALSE) |>
#     rename(age = X1, sex = X2, cp = X3, trestbps = X4, chol = X5, fbs = X6, restecg = X7, thalach = X8, exang = X9, 
#            oldpeak = X10, slope = X11, ca = X12, thal = X13, num_predicted = X14)
# data

set.seed(1)
# reading the dataframe from URL, assigning col names and types
hungary_data <- read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.hungarian.data",
                           col_names = c("age", "sex", "cp", "trestbps", "chol", "fbs", "restecg", 
                                         "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"),
                           col_types = list("d", "d", "d", "d", "d", "d", "d", "d", "d", "d", "d", "f", "f", "d"))
hungary_data
#dataframe be used
heart_data <- data |>
    select(age, sex, cp, trestbps, num_predicted)

#changed num to factor type
heart_data <- heart_data |>
    mutate(num_predicted = as_factor(num_predicted))
heart_data

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”


age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>
28,1,2,130,132,0,2,185,0,0,NA,?,?,0
29,1,2,120,243,0,0,160,0,0,NA,?,?,0
29,1,2,140,NA,0,0,170,0,0,NA,?,?,0
30,0,1,170,237,0,1,170,0,0,NA,?,6,0
31,0,2,100,219,0,1,150,0,0,NA,?,?,0
32,0,2,105,198,0,0,165,0,0,NA,?,?,0
32,1,2,110,225,0,0,184,0,0,NA,?,?,0
32,1,2,125,254,0,0,155,0,0,NA,?,?,0
33,1,3,120,298,0,0,185,0,0,NA,?,?,0


age,sex,cp,trestbps,num_predicted
<dbl>,<dbl>,<dbl>,<chr>,<fct>
32,1,1,95,1
34,1,4,115,1
35,1,4,?,3
36,1,4,110,1
38,0,4,105,2
38,0,4,110,1
38,1,3,100,0
38,1,3,115,1
38,1,4,135,2


In [2]:
#split data into training and testing
heart_split <- initial_split(heart_data, prop = 0.75, strata = num_predicted)
heart_train <- training(heart_split)
heart_test <- testing(heart_split)